We begin by importing required Python libraries for data manipulation, preprocessing, and dimensionality reduction (PCA). We also import defaultdict to help with hash table creation in later steps.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from collections import defaultdict


We load the FMA metadata and filter the dataset to include only tracks from the "medium" subset. We then keep only the 8 most relevant genres and scale the feature values using StandardScaler. Additionally, we apply PCA to reduce the feature space to 50 components. Finally, we split the data into training, validation, and test sets.

In [12]:
#  Load metadata (tracks)
df_tracks = pd.read_csv('tracks.csv', index_col=0, header=[0, 1])
df_tracks = df_tracks[df_tracks[('set', 'subset')] == 'medium']

# Filter for 8 genres
genres = ['Hip-Hop', 'Pop', 'Folk', 'Rock', 'Experimental', 'International', 'Electronic', 'Instrumental']
df_tracks = df_tracks[df_tracks[('track', 'genre_top')].isin(genres)]

#  simplified split and genre columns to add
df_tracks['split'] = df_tracks[('set', 'split')]
df_tracks['genre'] = df_tracks[('track', 'genre_top')]

#  Load features 
df_features = pd.read_csv('features.csv', index_col=0, header=[0, 1, 2])
df_features.columns = ['_'.join(col).strip() for col in df_features.columns.values]

#  USE/store/keep  only the tracks from filtered metadata
df_features = df_features.loc[df_tracks.index]

#  Create Feature Sets from the three options

# a) Kurtosis only
kurtosis_cols = [col for col in df_features.columns if 'kurtosis' in col]
X_kurtosis = df_features[kurtosis_cols]

# b) All features
X_all = df_features.copy()

# c) PCA-reduced (50 components)
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)
pca = PCA(n_components=50)
X_pca = pd.DataFrame(pca.fit_transform(X_all_scaled), index=X_all.index)

#  The Final Splits 
y = df_tracks['genre']

splits = {
    'train': df_tracks[df_tracks['split'] == 'training'],
    'valid': df_tracks[df_tracks['split'] == 'validation'],
    'test': df_tracks[df_tracks['split'] == 'test']
}

# Now we store all the data 
data = {}
X_dict = {
    'kurtosis': X_kurtosis,
    'all': X_all,
    'pca': X_pca
}

for key in X_dict.keys():
    data[key] = {}
    for split_name, df_split in splits.items():
        idx = df_split.index
        data[key][split_name] = {
            'X': X_dict[key].loc[idx],
            'y': y.loc[idx]
        }

# Extract arrays from 'data' to match the variable names from second snippet
X_all_train = data['all']['train']['X'].values
X_pca_train = data['pca']['train']['X'].values
y_train = data['all']['train']['y'].values

X_all_val = data['all']['valid']['X'].values
X_pca_val = data['pca']['valid']['X'].values
y_val = data['all']['valid']['y'].values

X_all_test = data['all']['test']['X'].values
X_pca_test = data['pca']['test']['X'].values
y_test = data['all']['test']['y'].values

print(" Data prepared with kurtosis, all, and PCA feature sets for training, validation, and test.")

 Data prepared with kurtosis, all, and PCA feature sets for training, validation, and test.


## TASK 1 

For each track in the (standardized) training data, calculate its hash value of length l using LSH. Do so for n different hash tables and add the track to its corresponding bucket in each hash table.

This function creates a random matrix following the Achlioptas method. Each entry is selected from {√3, 0, -√3} with specific probabilities to form a sparse random projection matrix.

In [24]:
def generate_random_matrix(l, d): #the random matrix
    probs = np.random.rand(l, d)
    R = np.zeros((l, d))
    R[probs < 1/6] = np.sqrt(3)
    R[(probs >= 1/6) & (probs < 5/6)] = 0
    R[probs >= 5/6] = -np.sqrt(3)
    return R

def compute_hash(R, x):
    return tuple((np.dot(R, x) > 0).astype(int))


This function constructs n LSH tables by projecting training data using randomly generated matrices. Each data point is hashed into a bucket based on the sign of its projections.

In [5]:
def build_lsh_tables(X_train, n_tables=5, l=16): #lsh function 
    d = X_train.shape[1]
    hash_tables = []
    projection_matrices = []

    for _ in range(n_tables):
        R = generate_random_matrix(l, d)
        table = defaultdict(list)
        for idx, x in enumerate(X_train):
            h = compute_hash(R, x)
            table[h].append(idx)
        hash_tables.append(table)
        projection_matrices.append(R)

    return hash_tables, projection_matrices


We now apply the build_lsh_tables function to the training set with n_tables=5 and hash length l=16. This step generates the necessary hash tables and projection matrices for approximate nearest neighbor search.

In [14]:
hash_tables_all, proj_all = build_lsh_tables(X_all_train, n_tables=5, l=16)


In [16]:
hash_tables_pca, proj_pca = build_lsh_tables(X_pca_train, n_tables=5, l=16)


Then to proceed with Task 2 ,we agreed to first try all three options and choose the best peroforming one.

In [18]:
import numpy as np
from collections import defaultdict

class LSH:
    def __init__(self, input_dim, l=10, n_tables=5, seed=None):
        np.random.seed(seed)
        self.l = l
        self.n_tables = n_tables
        self.input_dim = input_dim
        self.hash_tables = [defaultdict(list) for _ in range(n_tables)]
        self.random_matrices = [self._generate_random_matrix() for _ in range(n_tables)]

    def _generate_random_matrix(self):
        # Achlioptas random projection matrix
        prob_matrix = np.random.choice([np.sqrt(3), 0, -np.sqrt(3)], 
                                       size=(self.input_dim, self.l), 
                                       p=[1/6, 2/3, 1/6])
        return prob_matrix

    def _hash(self, R, x):
        projection = np.dot(x, R)
        return ''.join(['1' if val >= 0 else '0' for val in projection])

    def fit(self, X):
        # X: DataFrame or numpy matrix (rows are samples)
        for table_idx in range(self.n_tables):
            R = self.random_matrices[table_idx]
            for idx, x in X.iterrows():
                hash_code = self._hash(R, x.values)
                self.hash_tables[table_idx][hash_code].append(idx)

    def get_candidates(self, x):
        # Return candidate similar indices from all tables
        candidates = set()
        for table_idx in range(self.n_tables):
            R = self.random_matrices[table_idx]
            hash_code = self._hash(R, x.values)
            candidates.update(self.hash_tables[table_idx].get(hash_code, []))
        return list(candidates)


In [20]:
X_train = data['kurtosis']['train']['X']

# Initialize LSH with l=10 bits, n=5 tables
lsh = LSH(input_dim=X_train.shape[1], l=10, n_tables=5, seed=42)

# Fit on training data
lsh.fit(X_train)

# Example: get similar track candidates for one validation track
x_val = data['kurtosis']['valid']['X'].iloc[0]
candidates = lsh.get_candidates(x_val)
print("Found similar tracks:", candidates)


Found similar tracks: [32768, 40962, 8205, 57381, 57395, 57396, 57399, 57400, 131128, 57403, 139323, 57404, 57405, 57410, 8260, 139333, 32842, 122958, 8280, 41056, 57441, 57443, 57446, 57447, 106609, 106610, 98418, 131192, 57470, 82052, 16528, 16529, 82064, 57493, 57495, 8358, 65706, 16556, 24750, 198, 139487, 57568, 139488, 33017, 257, 8451, 73990, 106772, 106773, 90408, 8490, 8491, 57642, 57644, 98614, 16695, 16703, 8514, 24904, 24913, 106834, 8531, 131416, 65880, 16733, 65887, 147808, 147810, 33129, 131438, 106864, 131443, 106879, 24971, 33185, 33192, 425, 16826, 65979, 106940, 33218, 131531, 16848, 131537, 57812, 74198, 90588, 41442, 82405, 82406, 90631, 107019, 33301, 66069, 535, 33304, 537, 33306, 123421, 123423, 123426, 16940, 33333, 66124, 66125, 123469, 123471, 605, 82528, 90721, 82531, 635, 74364, 57989, 66182, 66186, 57995, 57998, 57999, 90769, 49811, 25236, 49813, 665, 115358, 115360, 115364, 148134, 90796, 115372, 90800, 115378, 90805, 98997, 131766, 99001, 33467, 115389, 

### TASK 2 

For each track $t_i$ in the validation data, find similar music tracks using LSH. A music track is defined as similar if it is in the same bucket as $t_i$ in one of the n hash tables.

In [29]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances
from collections import Counter

# Evaluation function for this task 
def evaluate_lsh_classifier(X_val, y_val, X_train, y_train, l=16, n_tables=5, k=5, metric='cosine'):
    lsh = LSH(input_dim=X_train.shape[1], l=l, n_tables=n_tables, seed=42)
    lsh.fit(X_train)

    correct = 0
    total = 0

    for idx, x_val in X_val.iterrows():
        candidates = lsh.get_candidates(x_val)
        if not candidates:
            continue

        X_candidates = X_train.loc[candidates]
        y_candidates = y_train.loc[candidates]

        # Compute distances
        if metric == 'cosine':
            dists = cosine_distances([x_val.values], X_candidates.values)[0]
        elif metric == 'euclidean':
            dists = euclidean_distances([x_val.values], X_candidates.values)[0]
        else:
            raise ValueError("Unknown metric")

        nearest_indices = np.argsort(dists)[:k]
        top_k_labels = y_candidates.iloc[nearest_indices]

        prediction = Counter(top_k_labels).most_common(1)[0][0]

        if prediction == y_val.loc[idx]:
            correct += 1
        total += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy, total

# Evaluate all three variants
results = []
for variant in ['kurtosis', 'all', 'pca']:
    for metric in ['cosine', 'euclidean']:
        X_train = data[variant]['train']['X']
        y_train = data[variant]['train']['y']
        X_val = data[variant]['valid']['X']
        y_val = data[variant]['valid']['y']

        acc, total = evaluate_lsh_classifier(X_val, y_val, X_train, y_train,
                                             l=16, n_tables=5, k=5, metric=metric)
        results.append((variant, metric, acc, total))

import pandas as pd
df_results = pd.DataFrame(results, columns=['Feature Set', 'Metric', 'Validation Accuracy', 'Evaluated Samples'])
print(df_results)



  Feature Set     Metric  Validation Accuracy  Evaluated Samples
0    kurtosis     cosine             0.567951               1479
1    kurtosis  euclidean             0.559838               1479
2         all     cosine             0.610033               1495
3         all  euclidean             0.608027               1495
4         pca     cosine             0.591510               1437
5         pca  euclidean             0.593598               1437


In [31]:
import itertools


from collections import defaultdict, Counter

Given a query vector x, this function(def find_candidate_neighbors) searches through the LSH tables to find candidate neighbors. It hashes the query using the same projection matrices and retrieves all items from the corresponding buckets across all hash tables.
Then After retrieving candidate neighbors, this function determines the genre of a track by performing a majority vote among the k nearest neighbors. It considers the genres of the closest tracks to predict the genre of the query track.
And Finally the function (evaluate_on_validation_set)assesses the performance of the LSH-based classifier by predicting genres for the validation set and comparing them to the true labels. It computes the accuracy of the predictions to gauge the effectiveness of the model.

In [33]:
def build_lsh_tables(X_train, n_tables=5, l=16, seed=42):
    np.random.seed(seed)
    input_dim = X_train.shape[1]
    hash_tables = [defaultdict(list) for _ in range(n_tables)]
    projection_matrices = [
        np.random.choice([np.sqrt(3), 0, -np.sqrt(3)],
                         size=(input_dim, l), p=[1/6, 2/3, 1/6])
        for _ in range(n_tables)
    ]
    for table_idx in range(n_tables):
        R = projection_matrices[table_idx]
        for idx, x in X_train.iterrows():
            proj = np.dot(x.values, R)
            hash_code = ''.join(['1' if val >= 0 else '0' for val in proj])
            hash_tables[table_idx][hash_code].append(idx)
    return hash_tables, projection_matrices

def find_candidate_neighbors(x_val, hash_tables, projection_matrices):
    candidates = set()
    for table_idx in range(len(hash_tables)):
        R = projection_matrices[table_idx]
        proj = np.dot(x_val.values, R)
        hash_code = ''.join(['1' if val >= 0 else '0' for val in proj])
        candidates.update(hash_tables[table_idx].get(hash_code, []))
    return list(candidates)

def predict_genre_for_track(x_val, X_train, y_train, hash_tables, projection_matrices,
                            k=5, metric='cosine', fallback_genre='Rock'):
    candidate_idxs = find_candidate_neighbors(x_val, hash_tables, projection_matrices)
    if not candidate_idxs:
        return fallback_genre
    X_candidates = X_train.loc[candidate_idxs]
    y_candidates = y_train.loc[candidate_idxs]
    if metric == 'cosine':
        dists = cosine_distances([x_val.values], X_candidates.values)[0]
    elif metric == 'euclidean':
        dists = euclidean_distances([x_val.values], X_candidates.values)[0]
    else:
        raise ValueError("Unknown metric")
    nearest_indices = np.argsort(dists)[:min(k, len(candidate_idxs))]
    top_k_labels = y_candidates.iloc[nearest_indices]
    return Counter(top_k_labels).most_common(1)[0][0]

def evaluate_on_validation_set(X_train, y_train, X_val, y_val, l, n_tables, k, metric):
    fallback_genre = y_train.mode()[0]
    hash_tables, projection_matrices = build_lsh_tables(X_train, n_tables=n_tables, l=l)
    correct = 0
    total = 0
    for idx, x in X_val.iterrows():
        pred = predict_genre_for_track(x, X_train, y_train,
                                       hash_tables, projection_matrices,
                                       k=k, metric=metric,
                                       fallback_genre=fallback_genre)
        if pred == y_val.loc[idx]:
            correct += 1
        total += 1
    acc = correct / total if total > 0 else 0
    return acc, total

Here we performed a Grid Search.In this section, we systematically experiment with various combinations of hyperparameters (l, n_tables, k) and similarity metrics (cosine, euclidean) to identify the configuration that yields the highest validation accuracy. 


In [37]:
import time

start_time = time.time()

l_values = [10, 16, 24]
n_values = [3, 5, 8]
k_values = [3, 5, 10]
metrics = ['cosine', 'euclidean']
param_grid = list(itertools.product(l_values, n_values, k_values, metrics))

# Run validation for all feature sets 
val_results = []

for variant in ['kurtosis', 'all', 'pca']:
    print(f"\n🔍 Evaluating feature set: {variant.upper()}")
    X_train = data[variant]['train']['X']
    y_train = data[variant]['train']['y']
    X_val = data[variant]['valid']['X']
    y_val = data[variant]['valid']['y']

    for l, n_tables, k, metric in param_grid:
        acc, total = evaluate_on_validation_set(X_train, y_train, X_val, y_val,
                                                l=l, n_tables=n_tables, k=k, metric=metric)
        val_results.append((variant, l, n_tables, k, metric, acc, total))

# Create The results to sort them 
df_val_results = pd.DataFrame(val_results, columns=[
    'Feature Set', 'l', 'n_tables', 'k', 'Metric', 'Validation Accuracy', 'Evaluated Samples'
])
df_val_results = df_val_results.sort_values(by='Validation Accuracy', ascending=False)

#  Confirm everything ran...I had a problem with the loop and only the option with the best performance run so i needed to add this line 
print("\n Validation runs per feature set:")
print(df_val_results['Feature Set'].value_counts())  # Should be 54 each

#  Show best config per feature set
for variant in ['kurtosis', 'all', 'pca']:
    print(f"\n Top results for {variant.upper()}:")
    display(df_val_results[df_val_results['Feature Set'] == variant].sort_values(by='Validation Accuracy', ascending=False).head(5)) 

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


🔍 Evaluating feature set: KURTOSIS

🔍 Evaluating feature set: ALL

🔍 Evaluating feature set: PCA

 Validation runs per feature set:
Feature Set
pca         54
all         54
kurtosis    54
Name: count, dtype: int64

 Top results for KURTOSIS:


,Feature Set,l,n_tables,k,Metric,Validation Accuracy,Evaluated Samples
16,kurtosis,10,8,10,cosine,0.597324,1495
34,kurtosis,16,8,10,cosine,0.588629,1495
10,kurtosis,10,5,10,cosine,0.585284,1495
17,kurtosis,10,8,10,euclidean,0.585284,1495
35,kurtosis,16,8,10,euclidean,0.576589,1495



 Top results for ALL:


,Feature Set,l,n_tables,k,Metric,Validation Accuracy,Evaluated Samples
59,all,10,3,10,euclidean,0.620736,1495
77,all,16,3,10,euclidean,0.620736,1495
100,all,24,5,10,cosine,0.620067,1495
94,all,24,3,10,cosine,0.620067,1495
106,all,24,8,10,cosine,0.620067,1495



 Top results for PCA:


,Feature Set,l,n_tables,k,Metric,Validation Accuracy,Evaluated Samples
124,pca,10,8,10,cosine,0.703010,1495
119,pca,10,5,10,euclidean,0.698997,1495
118,pca,10,5,10,cosine,0.694314,1495
125,pca,10,8,10,euclidean,0.692308,1495
122,pca,10,8,5,cosine,0.684950,1495


Elapsed time: 1726.35 seconds


Now to Complete Task Two, the best combination I used it for this next step:

In [38]:
# Load the best configuration
X_train = data['pca']['train']['X']
X_val = data['pca']['valid']['X']

# Step 1: Build hash tables
hash_tables, projection_matrices = build_lsh_tables(X_train, n_tables=8, l=10)

# Step 2: Find similar tracks for each validation track
similar_tracks_dict = {}

for idx, x_val in X_val.iterrows():
    similar = find_candidate_neighbors(x_val, hash_tables, projection_matrices)
    similar_tracks_dict[idx] = similar

# Step 3: Print examples
for track_id, neighbors in list(similar_tracks_dict.items())[:5]:  # only print first 5
    print(f"Track {track_id} has {len(neighbors)} similar tracks (same bucket):")
    print(neighbors[:10])  # Show up to 10 neighbors
    print("---")


Track 458 has 110 similar tracks (same bucket):
[76293, 144397, 117774, 93725, 144422, 56885, 3638, 125496, 57410, 32839]
---
Track 583 has 249 similar tracks (same bucket):
[82440, 71689, 26633, 1548, 117774, 1551, 8209, 14871, 132121, 14874]
---
Track 584 has 300 similar tracks (same bucket):
[151045, 109063, 137736, 144391, 13834, 144393, 144397, 14862, 151053, 89617]
---
Track 585 has 353 similar tracks (same bucket):
[61440, 14347, 38954, 94256, 69683, 43062, 69686, 139333, 67654, 67659]
---
Track 603 has 280 similar tracks (same bucket):
[19457, 70658, 76295, 6664, 32776, 15369, 37388, 123917, 126991, 126483]
---


### TASK 3 

Calculate the actual similarity of $t_i$ to all similar music tracks based on their feature vectors and some similarity metric m. As a genre prediction for ti, use the majority genre of its k-nearest-neighbours defined as the k most similar music tracks to $t_i$ as ranked by m found in the same bucket as $t_i$.

Note: Think about how to treat cases where there are less then k music tracks similar to $t_i$.

In [40]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances
from collections import Counter

def predict_genre_for_track(x_val, X_train, y_train, hash_tables, projection_matrices,
                            k=10, metric='cosine', fallback_genre='Rock'):
    candidate_idxs = find_candidate_neighbors(x_val, hash_tables, projection_matrices)

    if not candidate_idxs:
        return fallback_genre

    X_candidates = X_train.loc[candidate_idxs]
    y_candidates = y_train.loc[candidate_idxs]

    # Compute similarity
    if metric == 'cosine':
        dists = cosine_distances([x_val.values], X_candidates.values)[0]
    elif metric == 'euclidean':
        dists = euclidean_distances([x_val.values], X_candidates.values)[0]
    else:
        raise ValueError("Invalid metric")

    # Rank and get top k (or all if fewer than k)
    nearest_indices = np.argsort(dists)[:min(k, len(candidate_idxs))]
    top_k_labels = y_candidates.iloc[nearest_indices]

    # Majority vote
    predicted_genre = Counter(top_k_labels).most_common(1)[0][0]
    return predicted_genre


In [46]:
# Load best config
X_train = data['pca']['train']['X']
y_train = data['pca']['train']['y']
X_val = data['pca']['valid']['X']
y_val = data['pca']['valid']['y']
l, n_tables, k, metric = 10, 8, 10, 'cosine'
fallback_genre = y_train.mode()[0]

# Build hash tables from training data
hash_tables, projection_matrices = build_lsh_tables(X_train, n_tables=n_tables, l=l)

# Predict for all validation samples
correct = 0
total = 0

for idx, x_val in X_val.iterrows():
    pred = predict_genre_for_track(x_val, X_train, y_train,
                                   hash_tables, projection_matrices,
                                   k=k, metric=metric,
                                   fallback_genre=fallback_genre)
    if pred == y_val.loc[idx]:
        correct += 1
    total += 1

val_acc = correct / total
print(f" Validation accuracy using The method on task 2 : {val_acc:.4f} ({correct}/{total})")

 Validation accuracy using The method on task 2 : 0.7030 (1051/1495)


We apply the evaluate_accuracy function to the validation dataset to measure how well our LSH-based classifier performs.And we checked the hyperparameter tuning parameters.

In [48]:
# Use best performing config from Task 2
X_train = data['pca']['train']['X']
y_train = data['pca']['train']['y']
X_val = data['pca']['valid']['X']
y_val = data['pca']['valid']['y']

l = 10
n_tables = 8
k = 10
metric = 'cosine'
fallback_genre = y_train.mode()[0] #This are the best parameters/combinations

# Build LSH hash tables
hash_tables, projection_matrices = build_lsh_tables(X_train, n_tables=n_tables, l=l)

# Run prediction on validation set
correct = 0
total = 0

for idx, x_val in X_val.iterrows():
    pred = predict_genre_for_track(x_val, X_train, y_train,
                                   hash_tables, projection_matrices,
                                   k=k, metric=metric,
                                   fallback_genre=fallback_genre)
    if pred == y_val.loc[idx]:
        correct += 1
    total += 1

val_acc = correct / total
print(f" Final validation accuracy: {val_acc:.4f} ({correct}/{total} correct)")

 Final validation accuracy: 0.7030 (1051/1495 correct)


### Task 4 

Evaluate the classification accuracy of your algorithm on the validation data.

**Training the final model with optimal hyperparameters **
After determining the best hyperparameter configuration, we retrain the LSH model using both the training and validation datasets. This approach leverages all available labeled data to enhance the model's generalization ability before evaluating it on the test set.

In [50]:
# Combine training + validation for final training set
X_train_val = pd.concat([data['pca']['train']['X'], data['pca']['valid']['X']])
y_train_val = pd.concat([data['pca']['train']['y'], data['pca']['valid']['y']])

# Test set
X_test = data['pca']['test']['X']
y_test = data['pca']['test']['y']

# Best parameters
l = 10
n_tables = 8
k = 10
metric = 'cosine'
fallback_genre = y_train_val.mode()[0]

# Build final LSH hash tables
hash_tables, projection_matrices = build_lsh_tables(X_train_val, n_tables=n_tables, l=l)

# Evaluate on test set
correct = 0
total = 0

for idx, x_test in X_test.iterrows():
    pred = predict_genre_for_track(x_test, X_train_val, y_train_val,
                                   hash_tables, projection_matrices,
                                   k=k, metric=metric,
                                   fallback_genre=fallback_genre)
    if pred == y_test.loc[idx]:
        correct += 1
    total += 1

test_acc = correct / total
print(f" Final test set accuracy: {test_acc:.4f} ({correct}/{total} correct)")


 Final test set accuracy: 0.6853 (1052/1535 correct)


**Testing the final LSH model **
We assess the performance of our optimized LSH-based classifier on the test dataset to estimate its generalization to unseen data. The test accuracy serves as an indicator of the model's effectiveness in real-world scenarios.